In [8]:
import requests
import pandas as pd
from textblob import TextBlob

# Function to perform sentiment analysis
def get_sentiment(text):
    if text:
        blob = TextBlob(text)
        sentiment_score = blob.sentiment.polarity
        if sentiment_score > 0:
            return 'Positive'
        elif sentiment_score < 0:
            return 'Negative'
        else:
            return 'Neutral'
    else:
        return 'Neutral'

# Function to fetch news articles related to stock symbols
def get_stock_news(api_key, stock_symbols):
    base_url = "https://newsapi.org/v2/everything"
    all_news = []

    for stock_symbol in stock_symbols:
        params = {
            "q": stock_symbol,
            "apiKey": api_key
        }

        try:
            response = requests.get(base_url, params=params)
            data = response.json()

            if response.status_code == 200 and data.get("status") == "ok":
                articles = data.get("articles", [])
                for article in articles:
                    title = article.get("title", "")
                    description = article.get("description", "")
                    if title and description:
                        news_entry = {
                            "Stock Symbol": stock_symbol,
                            "Title": title,
                            "Description": description,
                            "URL": article.get("url", ""),
                            "Published At": article.get("publishedAt", ""),
                            "Sentiment": get_sentiment(title + " " + description)
                        }
                        all_news.append(news_entry)
            else:
                print(f"Failed to retrieve news for {stock_symbol}. Error: {data.get('message', 'Unknown error')}")
        except Exception as e:
            print(f"Failed to retrieve news for {stock_symbol}. Error: {e}")

    return all_news

# Function to check sentiment of a particular stock symbol
def check_stock_sentiment(stock_symbol, df):
    stock_data = df[df['Stock Symbol'] == stock_symbol]
    if not stock_data.empty:
        overall_sentiment = stock_data['Sentiment'].value_counts().idxmax()
        print(f"Overall sentiment for {stock_symbol}: {overall_sentiment}")
    else:
        print(f"No news found for {stock_symbol}")

# Load API key from a file (assuming 'apikey' file exists with API key)
api_key = open('apikey', 'r').read()

# Define stock symbols to fetch news for
stock_symbols = ['MSFT', 'AAPL', 'NVDA', 'AMZN', 'GOOG', 'GOOGL', 'META', 'BRK/A', 'BRK/B', 'LLY']

# Fetch news articles for the specified stock symbols
stock_news = get_stock_news(api_key, stock_symbols)

# Create DataFrame from the fetched news data
df = pd.DataFrame(stock_news)

# Print instructions for user input
print("Enter a stock symbol to check its sentiment:")
user_input = input().upper()

# Check sentiment for the specified stock symbol
check_stock_sentiment(user_input, df)

Enter a stock symbol to check its sentiment:


 AAPL


Overall sentiment for AAPL: Positive


In [2]:
from azure.storage.blob import BlobServiceClient

# Function to upload CSV file to Azure Blob Storage
def upload_to_azure_blob(csv_file, container_name, blob_name, connection_string):
    try:
        blob_service_client = BlobServiceClient.from_connection_string(connection_string)
        container_client = blob_service_client.get_container_client(container_name)
        blob_client = container_client.get_blob_client(blob_name)
        
        with open(csv_file, "rb") as data:
            blob_client.upload_blob(data, overwrite=True)
            print(f"Uploaded to Azure Blob Storage in container '{container_name}' as '{blob_name}'.")
    except Exception as e:
        print(f"An error occurred during upload to Azure Blob Storage: {e}")

In [7]:
# # Export the DataFrame to a CSV file
# output_csv_file = "sentiment_analysis_results.csv"
# df.to_csv(output_csv_file, index=False)

# print(f"Sentiment analysis results have been exported to {output_csv_file}.")

In [6]:
# Upload the CSV file to Azure Blob Storage
container_name = "analysis4"  # Replace with your actual container name
blob_name = "sentiment_analysis_results.csv"
connection_string = "DefaultEndpointsProtocol=https;AccountName=sentiment4;AccountKey=FZwusBEA1ckXRH8EOrGQ3dxC2SUgSOY8rT4fE3CGRgHep5+7hyzqnxfmL2nmbAmtcArcM5C0zZaS+AStusyS9g==;EndpointSuffix=core.windows.net"  # Replace with your actual connection string
upload_to_azure_blob(output_csv_file, container_name, blob_name, connection_string)

print(f"Sentiment analysis results have been exported to {output_csv_file} and uploaded to Azure Blob Storage.")


Uploaded to Azure Blob Storage in container 'analysis4' as 'sentiment_analysis_results.csv'.
Sentiment analysis results have been exported to sentiment_analysis_results.csv and uploaded to Azure Blob Storage.
